In [ ]:
import pandas as pd
import yfinance as yf
from sqlalchemy import create_engine

In [ ]:
engine = create_engine('sqlite:///Section 14 Video 72 SNP 500 Stocks DataBase.db')

### Database management functions proof of concept

In [ ]:
df = yf.download('AAPL', start= '2010-01-01', end= '2024-08-31')
df

In [ ]:
df.to_sql('Apple', engine)

In [ ]:
pd.read_sql('Apple', engine)

In [ ]:
df_2 = yf.download('AAPL', start= '2010-01-01', end= '2024-09-06')
df_2
#there is an additional week's worth of data but the rest of the df is teh same

In [ ]:
max_date = pd.read_sql('Select max(Date) from Apple', engine).values[0][0]

In [ ]:
df_2[df_2.index > max_date]

### Functions to Abstract Database Management 

In [ ]:
def custom_create_engine(name :str):
    engine = create_engine(f'sqlite:///{name} DataBase.db')
    return engine 

def create_db(ticker: str,start: str):
    df = yf.download(ticker, pd.to_datetime(start))
    df.to_sql(f'{ticker}', engine)
    return pd.read_sql(f'{ticker}', engine)

def SQL_importer(ticker, engine):
    try:    
        #updating a table already in DB
        max_date = pd.read_sql(f'Select max(Date) from {ticker}', engine).values[0][0]
        df = yf.download(ticker, start = pd.to_datetime(max_date))
        new_values = df[df.index > max_date]
        new_values.to_sql(f'{ticker}', if_exists = 'append')
        return print(f'{len(new_values)} new rows have been added since the last date of {max_date} in the database') 
    except:
        #create the table in DB if it doesn't exist
        create_db(ticker,'2010-01-01')
        print(f'{ticker} data was imported from analysis start to {max_date}')

In [ ]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol']
tickers = tickers.to_list()

In [ ]:
create_db(tickers[0],'2010-01-01')

In [ ]:
for ticker in tickers:
    try:
        SQL_importer(ticker,engine)
    except:
        print(f'import of {ticker} failed')

### ^^^Video 73 is teach's way of doing the above ^^^